In [1]:

%pip install vt-py

Note: you may need to restart the kernel to use updated packages.


In [2]:
#Import library
import numpy as np
import random as rm
import pandas as pd
import json
import matplotlib
import re
import csv
import time
import shutil
import math
from numpy.lib.shape_base import tile
from sklearn.preprocessing import normalize
import string
import re
from collections import defaultdict
from difflib import SequenceMatcher
import networkx as nx
from networkx.drawing.nx_agraph import to_agraph
import imageio
import requests
import urllib.request
import vt
import os

# JSON to pandas dataframe

In [7]:
def create_df(file):
  '''
  Creates pandas dataframe from raw json file.
  @ file: file path to raw json file.
  @ return: returns dataframe
  '''

  ### COLLINS: may need to change this dates. It is the trailing dates in the json files.
  # dates=['2024-08-30','2024-08-31','2024-09-01','2024-09-02','2024-09-03',
  #        '2024-09-04','2024-09-05','2024-09-06','2024-09-07','2024-09-08', '2024-09-09', '2024-09-10', '2024-09-17', '2024-09-18', '2024-09-19', '2024-09-20', '2024-09-21', '2024-09-22', '2024-09-23', '2024-09-24', '2024-09-25', '2024-09-26']
  dates = ["2024-10-02","2024-10-03","2024-10-04","2024-10-05","2024-10-06",
           "2024-10-07","2024-10-08","2024-10-09","2024-10-10","2024-10-11","2024-10-12",
           "2024-10-13","2024-10-14","2024-10-15"]

  df_list = []

  #df_temp = df_temp = pd.read_json(f'{file}.2024-08-29', lines=True)
  df_temp =pd.read_json(f'{file}.2024-10-01', lines=True)


  df_list.append(df_temp)

  for d in dates:
    file2 = f'{file}.{d}'
    print("doing", file2)
    try:
      df_temp = pd.read_json(file2, lines=True)
      df_list.append(df_temp)
    except:
      print("Error Occured (no need to care)")

  df_pending = pd.concat(df_list)
  df_pending['timestamp'] = pd.to_datetime(df_pending['timestamp'])
  #df_pending = df_pending[df_pending.sourceIp != "128.59.15.19"]

  df_pending.sort_values(by='timestamp', inplace = True)
  df_pending.drop(df_pending[df_pending['src_port'] == 0].index, inplace=True)
  df_pending.index = range(len(df_pending))
  return df_pending

In [10]:
# COLLINS: change the path
raw_pending = create_df('./raw_data/test_raw_data/pending/cowrie.json')
raw_ebilling = create_df('./raw_data/test_raw_data/ebilling/cowrie.json')
raw_crm = create_df('./raw_data/test_raw_data/csh/cowrie.json')


doing ./raw_data/test_raw_data/pending/cowrie.json.2024-10-02
doing ./raw_data/test_raw_data/pending/cowrie.json.2024-10-03
doing ./raw_data/test_raw_data/pending/cowrie.json.2024-10-04
doing ./raw_data/test_raw_data/pending/cowrie.json.2024-10-05
Error Occured (no need to care)
doing ./raw_data/test_raw_data/pending/cowrie.json.2024-10-06
Error Occured (no need to care)
doing ./raw_data/test_raw_data/pending/cowrie.json.2024-10-07
Error Occured (no need to care)
doing ./raw_data/test_raw_data/pending/cowrie.json.2024-10-08
doing ./raw_data/test_raw_data/pending/cowrie.json.2024-10-09
doing ./raw_data/test_raw_data/pending/cowrie.json.2024-10-10
doing ./raw_data/test_raw_data/pending/cowrie.json.2024-10-11
doing ./raw_data/test_raw_data/pending/cowrie.json.2024-10-12
doing ./raw_data/test_raw_data/pending/cowrie.json.2024-10-13
doing ./raw_data/test_raw_data/pending/cowrie.json.2024-10-14
doing ./raw_data/test_raw_data/pending/cowrie.json.2024-10-15
doing ./raw_data/test_raw_data/ebill

In [11]:
def create_clean_df(df):
  '''
  Takes a raw dataframe and deletes unnecessary columns.
  @ df: pandas dataframe (the raw dataframe returned by create_df)
  @ return: cleaned dataframe
  '''
  df_clean = pd.DataFrame()
  # COLLINS: modify this part (add the routines) if you want to include other columns from raw dataframe
  list_tf = list()
  list_ip=list()
  list_port = list()
  list_node = list()
  list_session = list()

  for i in range(len(df)):
    if df['eventid'][i] =="cowrie.command.input":
      cmd = df['message'][i][5:]
      if cmd=='':
        continue

      #list_tf.append(df['timestamp'][i])
      list_ip.append(df['src_ip'][i])
      #list_port.append(df['src_port'][i])
      list_session.append(df['session'][i])

      #list_node.append(cmd)

  #df_clean["timestamp"] = list_tf
  df_clean["src_ip"] = list_ip
  #df_clean["src_port"] = list_port
  df_clean["session"] = list_session

  #df_clean["node"] = list_node


  return df_clean

In [12]:
pending_clean = create_clean_df(raw_pending)
ebilling_clean = create_clean_df(raw_ebilling)
crm_clean = create_clean_df(raw_crm)



In [13]:
pending_clean.sort_values("src_ip", axis=0, ascending=True, inplace=True, na_position='last')
new_pending_clean = pending_clean.drop_duplicates(subset=['session'])
new_pending_clean

,src_ip,session
48107,1.175.50.115,38ceb172ad3e
57182,1.214.197.163,e3fc11f25f5e
58157,1.214.197.163,e98c1ea9d813
58037,1.214.197.163,c4c46603a7e2
58228,1.214.197.163,542c8de22379
...,...,...
45963,96.78.175.36,110b4c6f80a1
7738,96.78.175.39,44af9c8fa86c
7616,96.78.175.39,852c064e14ee
7539,96.78.175.39,8e941429e2e0


In [14]:
src_ip = list(set(list(new_pending_clean["src_ip"])))

In [ ]:
headers = {        
    'User-agent': 'yomamaur bot 0.1',
   # 'x-apikey': USE YOUR KEY HERE
}

response = requests.get(f'https://www.virustotal.com/api/v3/ip_addresses/103.177.212.2', headers=headers)
response_json = response.json()


In [16]:
response.status_code

200

In [17]:
temp_response = dict()

In [ ]:
import requests

def check_ip_on_virustotal(ip_address, api_keys):
    url = f'https://www.virustotal.com/api/v3/ip_addresses/{ip_address}'


    headers = {
        'User-agent': 'your bot 0.1',
        'x-apikey': api_keys[0]
    }
    counter = 0
    error_counter =0

    try:
        counter += 1
        if counter%200 == 0:
            print("sleeping for 30 seconds")
            time.sleep(30)
        response = requests.get(url, headers=headers)
        response_json = response.json()
        print(response.status_code)
        
        while response.status_code == 429:
            error_counter +=1
            print(f"out of quota. trying [{error_counter}]")
            headers = {
                f'User-agent': f'your bot {counter}',
                'x-apikey': api_keys[error_counter]
            }
            response = requests.get(url, headers=headers)
            response_json = response.json()
        
        if response.status_code == 200:
            data = response_json['data']

            if 'attributes' in data:
                attributes = data['attributes']
                country = attributes.get('country', 'Unknown')
                last_analysis_results = attributes.get('last_analysis_results')

                print("IP Address:", ip_address)
                #print("Country:", country)
                #print("Last Analysis Results:")

                analysis_dict = dict()
                analysis_dict['IP Address'] = ip_address
                analysis_dict['Country'] = country

                for engine, result in last_analysis_results.items():
                    if result['result'] in analysis_dict.keys():
                      analysis_dict[result['result']] += 1
                    else:
                      analysis_dict[result['result']] = 1

                print(analysis_dict)
                analysis_df = df = pd.DataFrame.from_records([analysis_dict])
                return analysis_dict
                #analysis_df.to_csv('analysis.csv')

                    #print(f"{engine}: {result['result']}")
            else:
                print("No information available for the IP address.")

        else:
            print("Error occurred while checking the IP address.")

    except requests.exceptions.RequestException as e:
        print("An error occurred during the request:", str(e))



# TODO: fill in VirusTotal API key
api_key_list =[]


In [19]:

def create_ip_result(clean_df):
  
  # drops duplicated sessions
  clean_df.sort_values("src_ip", axis=0, ascending=True, inplace=True, na_position='last')
  new_clean = clean_df.drop_duplicates(subset=['session'])
  new_clean.index = range(len(new_clean))
  
  
  df_ip_report = pd.DataFrame()

  list_ip = list()
  list_session = list()
  
  list_clean=list()
  list_unrated = list()
  list_suspicious = list()
  list_malicious = list()
  #list_malware = list()

  for i in range(len(new_clean)):
    curr_ip = new_clean["src_ip"][i]
    list_session.append(new_clean["session"][i])
    list_ip.append(curr_ip)
    if curr_ip in temp_response.keys() and temp_response[curr_ip] != None:
      response = temp_response[curr_ip]
    else:
      response= check_ip_on_virustotal(curr_ip, api_key_list)
      temp_response[curr_ip] = response
      
    print(response)
    if "clean" not in response:
      list_clean.append(0)
    else:
      list_clean.append(response["clean"])
      
    if "unrated" not in response:
      list_unrated.append(0)
    else:
      list_unrated.append(response["unrated"])
      
    if "suspicious" not in response:
      list_suspicious.append(0)
    else:
      list_suspicious.append(response["suspicious"])
       
    if "malicious" not in response:
      list_malicious.append(0)
    else:
      list_malicious.append(response["malicious"])
      
      
    
    #list_malware.append(response["malware"])

  df_ip_report["src_ip"] = list_ip
  df_ip_report["session"] = list_session
  df_ip_report["clean"] = list_clean
  df_ip_report["unrated"] = list_unrated
  df_ip_report["suspicious"] = list_suspicious
  df_ip_report["malicious"] = list_malicious
  #df_ip_report["malware"] = list_malware
  
  return df_ip_report
  

In [22]:
ip_report_pending = create_ip_result(pending_clean)

{'IP Address': '1.175.50.115', 'Country': 'TW', 'clean': 60, 'unrated': 31, 'malware': 2, 'malicious': 1}
{'IP Address': '1.214.197.163', 'Country': 'KR', 'clean': 51, 'unrated': 31, 'suspicious': 3, 'malware': 4, 'phishing': 2, 'malicious': 3}
{'IP Address': '1.214.197.163', 'Country': 'KR', 'clean': 51, 'unrated': 31, 'suspicious': 3, 'malware': 4, 'phishing': 2, 'malicious': 3}
{'IP Address': '1.214.197.163', 'Country': 'KR', 'clean': 51, 'unrated': 31, 'suspicious': 3, 'malware': 4, 'phishing': 2, 'malicious': 3}
{'IP Address': '1.214.197.163', 'Country': 'KR', 'clean': 51, 'unrated': 31, 'suspicious': 3, 'malware': 4, 'phishing': 2, 'malicious': 3}
{'IP Address': '1.214.197.163', 'Country': 'KR', 'clean': 51, 'unrated': 31, 'suspicious': 3, 'malware': 4, 'phishing': 2, 'malicious': 3}
{'IP Address': '1.214.197.163', 'Country': 'KR', 'clean': 51, 'unrated': 31, 'suspicious': 3, 'malware': 4, 'phishing': 2, 'malicious': 3}
{'IP Address': '1.215.240.130', 'Country': 'KR', 'clean': 54

In [23]:
with open("ip_temp_testing.json", "w") as f:
    json.dump(temp_response, f, indent=4)

In [24]:
ip_report_pending.to_csv('ip_report_pending_testing.csv', index=False)


In [34]:
ip_report_ebilling = create_ip_result(ebilling_clean)

{'IP Address': '1.178.45.115', 'Country': 'BR', 'clean': 58, 'unrated': 32, 'malicious': 2, 'suspicious': 1, 'malware': 1}
{'IP Address': '1.178.45.115', 'Country': 'BR', 'clean': 58, 'unrated': 32, 'malicious': 2, 'suspicious': 1, 'malware': 1}
{'IP Address': '1.232.29.63', 'Country': 'KR', 'clean': 58, 'unrated': 32, 'suspicious': 1, 'malware': 2, 'malicious': 1}
{'IP Address': '1.232.29.63', 'Country': 'KR', 'clean': 58, 'unrated': 32, 'suspicious': 1, 'malware': 2, 'malicious': 1}
{'IP Address': '1.232.29.63', 'Country': 'KR', 'clean': 58, 'unrated': 32, 'suspicious': 1, 'malware': 2, 'malicious': 1}
{'IP Address': '1.232.29.63', 'Country': 'KR', 'clean': 58, 'unrated': 32, 'suspicious': 1, 'malware': 2, 'malicious': 1}
{'IP Address': '1.232.29.63', 'Country': 'KR', 'clean': 58, 'unrated': 32, 'suspicious': 1, 'malware': 2, 'malicious': 1}
{'IP Address': '1.232.29.63', 'Country': 'KR', 'clean': 58, 'unrated': 32, 'suspicious': 1, 'malware': 2, 'malicious': 1}
{'IP Address': '1.232.

In [35]:
ip_report_ebilling.to_csv('ip_report_ebilling_testing.csv', index=False)


In [36]:
ip_report_crm = create_ip_result(crm_clean)

429
out of quota. trying [1]
out of quota. trying [2]
out of quota. trying [3]
IP Address: 1.14.139.92
{'IP Address': '1.14.139.92', 'Country': 'CN', 'clean': 56, 'unrated': 32, 'malicious': 2, 'suspicious': 1, 'phishing': 2, 'malware': 1}
{'IP Address': '1.14.139.92', 'Country': 'CN', 'clean': 56, 'unrated': 32, 'malicious': 2, 'suspicious': 1, 'phishing': 2, 'malware': 1}
{'IP Address': '1.14.139.92', 'Country': 'CN', 'clean': 56, 'unrated': 32, 'malicious': 2, 'suspicious': 1, 'phishing': 2, 'malware': 1}
429
out of quota. trying [1]
out of quota. trying [2]
out of quota. trying [3]
IP Address: 1.173.197.29
{'IP Address': '1.173.197.29', 'Country': 'TW', 'clean': 58, 'unrated': 32, 'suspicious': 1, 'malicious': 2, 'malware': 1}
{'IP Address': '1.173.197.29', 'Country': 'TW', 'clean': 58, 'unrated': 32, 'suspicious': 1, 'malicious': 2, 'malware': 1}
429
out of quota. trying [1]
out of quota. trying [2]
out of quota. trying [3]
IP Address: 1.175.7.173
{'IP Address': '1.175.7.173', 'Co

In [37]:
ip_report_crm.to_csv('ip_report_crm_testing.csv', index=False)
